In [1]:
import rpy2.robjects
from rpy2.robjects.packages import importr
import pandas as pd
from openpyxl import load_workbook

import warnings
warnings.filterwarnings('ignore')

In [2]:
markets = ['RU', 'UA', 'KZ', 'GE']
start, end = '2017-08-21', '2017-08-27'
year = 2017
campaign = 'C12'
week = 'Week 2'
segment_name_RU = '(C12-17) Exclude HIGH Fraud Orders'

In [3]:
%load_ext rpy2.ipython

In [4]:
importr('RSiteCatalyst', '.../R/win-library/3.4')
importr('curl', '...R/win-library/3.4')
importr('httr', '...R/win-library/3.4')

rpy2.robjects.packages.Package as a <module 'httr' (built-in)>

In [5]:
%Rpush start end segment_name_RU

In [6]:
%%R

#This line is needed to make RSiteCatalyst work in IPython
set_config( config( ssl_verifypeer = 0L ) )

#Authenticate into Sitecat API (Enter your Credentials Here)
SCAuth(login, pass)

suites <- c(suite1, suite2, suite3, suite4)

report_new <- data.frame()

for (suite in suites) {
  if (suite == 'avonrusdce-prod'){
    report <- QueueOvertime(suite, start, end, metrics = c('orders', 'revenue'),
                            segment.id = subset(GetSegments('avonrusdce-prod'), 
                                                name == segment_name_RU, 
                                                select = id),
                            date.granularity = 'day')
  } else {
    report <- QueueOvertime(suite, start, end, metrics = c('orders', 'revenue'),
                            date.granularity = 'day')
  }
  
  report_new <- rbind(report_new, report)
}

In [7]:
%Rpull report_new

In [8]:
x, y = 0, 7

for market in markets:
    df = pd.read_excel('..xlsx', sheetname=market)
    df.loc[(df.Year == year) & (df.Camp == campaign) & (df.Week == week), 'Ord.F'] = report_new.orders[x:y].sum()
    df.loc[(df.Year == year) & (df.Camp == campaign) & (df.Week == week), 'Rev.F'] = report_new.revenue[x:y].sum()
    df.loc[(df.Year == year) & (df.Camp == campaign) & (df.Week == week), 'AVO.F'] = report_new.revenue[x:y].sum() / report_new.orders[x:y].sum()
    
    book = load_workbook('..xlsx')
    writer = pd.ExcelWriter('..xlsx', engine='openpyxl') 
    writer.book = book
    writer.sheets = dict((ws.title, ws) for ws in book.worksheets)
    df.to_excel(writer, market, index=False)
    writer.save()
    
    x += 7
    y += 7